In [26]:
from dotenv import load_dotenv
load_dotenv()
import os
google_api_key = os.getenv('GOOGLE_API_KEY')
if not google_api_key: 
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")


In [27]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-flash-latest", model_provider="google_genai")

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template("""
Extract the desired information from the following passage.
Only extract the properties mentioned in the 'Classification' function.

Passage: {input}
"""
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: str = Field(description="How aggressive the text is on a scale from 1 to 10 ")
    language: str = Field(description="The language the text is written in")

structured_llm = llm.with_structured_output(Classification)

In [29]:
structured_llm

RunnableBinding(bound=ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, model='models/gemini-flash-latest', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x17d6e8770>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Classification', 'description': '', 'parameters': {'properties': {'sentiment': {'description': 'The sentiment of the text', 'type': 'string'}, 'aggressiveness': {'description': 'How aggressive the text is on a scale from 1 to 10 ', 'type': 'string'}, 'language': {'description'

In [30]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='Extremely positive', aggressiveness='1', language='Spanish')

In [31]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)
response.model_dump()

{'sentiment': 'Extremely Negative/Hostile',
 'aggressiveness': '10',
 'language': 'Spanish'}

In [32]:
# Finer controll
from typing import Literal
from pydantic import BaseModel, Field

class Classification(BaseModel):
    sentiment: Literal["happy", "neutral", "sad"]
    aggressiveness: Literal[1, 2, 3, 4, 5] = Field(
        description="describes how aggressive the statement is, the higher the number the more aggressive"
    )
    language: Literal["spanish", "english", "french", "german", "italian"]

llm2 = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


In [33]:

from langchain_core.output_parsers import PydanticOutputParser

# Create parser and prompt template
parser = PydanticOutputParser(pydantic_object=Classification)

tagging_prompt = ChatPromptTemplate.from_template(
    """Extract the desired information from the following passage and return ONLY a valid JSON object.

{format_instructions}

Passage: {input}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm2.invoke(prompt)
result = parser.parse(response.content)
result.model_dump()

{'sentiment': 'happy', 'aggressiveness': 1, 'language': 'spanish'}

In [34]:
# Create the chain using pipe operator
chain = tagging_prompt | llm2 | parser

# Usage
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
result = chain.invoke({"input": inp})
result.model_dump()

{'sentiment': 'happy', 'aggressiveness': 1, 'language': 'spanish'}

In [35]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
result = chain.invoke({"input": inp})
result.model_dump()

{'sentiment': 'happy', 'aggressiveness': 1, 'language': 'spanish'}

In [36]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
result = chain.invoke({"input": inp})
result.model_dump()

{'sentiment': 'sad', 'aggressiveness': 4, 'language': 'spanish'}

In [37]:
inp = "Weather is ok here, I can go outside without much more than a coat."
result = chain.invoke({"input": inp})
result.model_dump()

{'sentiment': 'neutral', 'aggressiveness': 1, 'language': 'english'}